In [1]:
import numpy as np
from itertools import combinations
from numpy import random
import math
import matplotlib.pyplot as plt
from scipy.stats import norm
import sample_hdx as hdx

In [2]:
# find idempotent
n = 10
l = math.comb(n, 3)
# res = np.zeros((l, l)) 
V = hdx.get_coord(n, 3)

# count = [math.comb(3, 3 - l) * math.comb(n - 3, l) for l in range(3 + 1)]
# print(count)

# each row is phi_i(j)
spherical_fn =  np.array([hdx.PHI(n, 3, i) for i in range(0, 4)])

# p number 
k_num = [hdx.k_i(n, 3, i) for i in range(0, 4)]
# print(k_num)
p_num = (spherical_fn * k_num).T
# get E_i
E =[]

idemp_val = hdx.idempotent_vals(n)
for i in range(0, 4):
    E.append(hdx.cov_mat(V, np.flip(idemp_val[i])))
# for i in range(0, 4):
#     cur_vales = spherical_fn[i]
    
# #     cur_eig = eigenval(n, 3, k)
# #     print(cur_eig)
    
# #     print(cur_vales)
#     dim = math.comb(n, i) 
#     if i > 0:
#         dim -= math.comb(n, i - 1)
    
#     cur = cov_mat(V, np.flip(cur_vales)) * dim/l
#     E.append(cur)

# get A    
for i in range(0, 4):
#     get A from indempotent 
    cur_p = p_num[i]
    cur_A = sum([cur_p[j] * E[j] for j in range(0, 4)])
    
#     compute A by hand
    test_A = np.zeros((l, l))
    for j in range(l):
        for k in range(j + 1):
            intersect = 3 - sum([v in V[j] for v in V[k]])
            if intersect == i:
                test_A[j][k] = 1
                test_A[k][j] = 1
#     print(cur_A)
    print('Adj matrix equal for dist ' + str(i), np.all(test_A == np.rint(cur_A)))
    
# check if E_i are basis 
print('sum E_i is I', np.all(np.rint(sum(E)) == np.identity(l)))
for i in range(0, 4):
    cur = E[i]
    print('E_i^2 = E_i E' + str(i), np.all(np.rint(cur.dot(cur) - cur) == np.zeros((l, l))))
    for j in range(i):
        print('orthogonal E' + str(i) + ',E' + str(j), np.all(np.rint(E[j].dot(cur)) == np.zeros((l, l))))
        print('max is', np.max(np.abs(E[j].dot(cur))))
    

Adj matrix equal for dist 0 True
Adj matrix equal for dist 1 True
Adj matrix equal for dist 2 True
Adj matrix equal for dist 3 True
sum E_i is I True
E_i^2 = E_i E0 True
E_i^2 = E_i E1 True
orthogonal E1,E0 True
max is 2.906735698698661e-17
E_i^2 = E_i E2 True
orthogonal E2,E0 True
max is 1.1757570226065026e-18
orthogonal E2,E1 True
max is 5.747717117069821e-17
E_i^2 = E_i E3 True
orthogonal E3,E0 True
max is 2.923903954052178e-18
orthogonal E3,E1 True
max is 1.9509277714441335e-17
orthogonal E3,E2 True
max is 3.5388358909926865e-16


In [3]:
# Combine p number and other values 
vals = [0, 0, 1/4, 1]
cov = hdx.cov_mat(V, vals)

eigen_val = p_num * np.flip(np.array(vals))[:, None]
# check if we get cov mat from linear combination of Ei

total_eig = np.sum(eigen_val, axis = 0)
res = sum([total_eig[i] * E[i] for i in range(4)])
# res = np.zeros((l, l))
# for i in range(0, 4):
#     cur_eig = eigen_val[i]
#     res += sum([cur_eig[j] * E[j] for j in range(4)])
diff = res - cov
print('check we get the correct eigenvalues', np.all(np.rint(diff) == np.zeros((l, l))), np.max(np.abs(diff)))

# sqrt_eig = np.sqrt(total_eig)
sqrt_eig = hdx.sqrt_cov(n, vals)
cov_sqrt = sum([sqrt_eig[i] * E[i] for i in range(4)])
# print(sqrt_eig)
diff2 = cov_sqrt.dot(cov_sqrt) - cov
print('check we get the correct eigenvalues', np.all(np.rint(diff2) == np.zeros((l, l))), np.max(np.abs(diff2)))
print('check PSD', np.all(np.linalg.cholesky(cov_sqrt.dot(cov_sqrt)) != None))

check we get the correct eigenvalues True 5.551115123125783e-17
check we get the correct eigenvalues True 1.7763568394002505e-15
check PSD True


In [4]:
# test sample function get correct covariance matrix
# by get each column of the sqrt covariance matrix used by sample_gaussian function
# and compare to the sqrt cov matrix from above
max_err = []
for i in range(1):
    cur_gauss = np.zeros(l)
    cur_gauss[i] = 1
    cur_col = hdx.sample_gaussian(n, vals, cur_gauss) 
    max_err.append(np.max(np.abs(cur_col - cov_sqrt[i])))
print(max(max_err))

0.0
